In [2]:
from datetime import datetime
from elasticsearch import Elasticsearch
import pprint
from gensim import corpora
from gensim.utils import simple_preprocess
import numpy as np
from gensim import models

es = Elasticsearch()

res = es.search(index="wnote", body={"query": {"match_all": {}}})

def map_source_to_content(hitData):
    key = 'rawTextSearch'
    if key in hitData['_source']:
        return hitData['_source']['rawTextSearch']
    else:
        return ''

documents = list(map(map_source_to_content, res['hits']['hits']))
print(len(documents))


10


In [4]:

import nltk
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def remove_special_character(text):
    words = nltk.word_tokenize(text)
    return [word for word in words if word not in stop_words and word.isalpha()]

texts = list(map(remove_special_character, documents))

print(texts[0])
# Count word frequencies

from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
dictionary = corpora.Dictionary(processed_corpus)
print(dictionary.token2id)

print('simple_preprocess:', simple_preprocess(documents[1]))
print('doc2bow:', dictionary.doc2bow(simple_preprocess(documents[1])))

corpus = [dictionary.doc2bow(simple_preprocess(line)) for line in documents]
# print('corpus:', corpus)

['Furthermore', 'several', 'functions', 'combined', 'In', 'case', 'one', 'optionally', 'choose', 'apply', 'function', 'document', 'matches', 'given', 'filtering', 'queryCopy', 'cURLView', 'Console', 'GET', 'query', 'query', 'boost', 'functions', 'filter', 'match', 'test', 'bar', 'weight', 'filter', 'match', 'test', 'cat', 'weight', 'max', 'multiply', 'Boost', 'whole', 'Function', 'score', 'list', 'supported', 'functions', 'The', 'scores', 'produced', 'filtering', 'query', 'function', 'matter']
{'Furthermore': 0, 'GET': 1, 'In': 2, 'The': 3, 'apply': 4, 'bar': 5, 'boost': 6, 'case': 7, 'cat': 8, 'choose': 9, 'combined': 10, 'document': 11, 'filter': 12, 'filtering': 13, 'function': 14, 'functions': 15, 'given': 16, 'match': 17, 'matches': 18, 'max': 19, 'multiply': 20, 'one': 21, 'optionally': 22, 'query': 23, 'score': 24, 'scores': 25, 'several': 26, 'test': 27, 'weight': 28, 'So': 29, 'component': 30, 'remove': 31, 'see': 32, 'want': 33, 'There': 34, 'information': 35, 'process': 36, 

In [7]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=8)
doc = 'furthermore queue run'
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]  # convert the query to LSI space
print(vec_lsi)

from gensim import similarities
index = similarities.MatrixSimilarity(lsi[corpus])  # transform corpus to LSI space and index it
sims = index[vec_lsi]  # perform a similarity query against the corpus
print('sims:', sims)
sims = sorted(enumerate(sims), key=lambda item: -item[1])
for i, s in enumerate(sims):
    print(i, s, documents[i][0:40])


[(0, 0.9275774074477878), (1, -0.2853434651071416), (2, -0.07842498755813812), (3, -0.04001312755472074), (4, 0.017894502326604216), (5, -0.004871871913983479), (6, 0.0035804786911264345), (7, -0.023521105204763186)]
sims: [ 3.2096729e-04 -5.0502829e-04 -3.4618657e-05  1.0082629e-03
 -5.7369471e-06  9.9992979e-01  1.5531015e-04 -2.0630658e-05
  9.0375543e-06  9.9711865e-01]
0 (5, 0.9999298) Furthermore, several functions can be co
1 (9, 0.99711865) Furthermore, several functions can be co
2 (3, 0.0010082629) Removing the Box Mesh
Lastly, we’ll hand
3 (0, 0.0003209673) There are two ways uncertainty intervene
4 (6, 0.00015531015) Ngoài cách dùng functions tính điểm ta c
5 (8, 9.037554e-06) Hostname to IP Address Lookup

This tool
6 (4, -5.736947e-06) test
adsssadd
erer66
sd hyhyh

asdasd
as
7 (7, -2.0630658e-05) The function_score query gives you the a
8 (2, -3.4618657e-05) Sorting in action

sdasdasdasd

Code dem
9 (1, -0.0005050283) Install middleware:

Postgres (> 11)Time


In [8]:
lda_model = models.ldamodel.LdaModel(
   corpus=corpus, id2word=dictionary, num_topics=5, random_state=100, 
   update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True
)
print(lda_model.print_topics())

[(0, '0.090*"score" + 0.061*"function" + 0.042*"functions" + 0.041*"component" + 0.032*"query" + 0.032*"remove" + 0.032*"results" + 0.031*"use" + 0.031*"define" + 0.022*"see"'), (1, '0.053*"điểm" + 0.036*"query" + 0.036*"mình" + 0.030*"những" + 0.030*"sẽ" + 0.030*"trả" + 0.030*"về" + 0.024*"shar" + 0.024*"sharland" + 0.024*"hoặc"'), (2, '0.009*"điểm" + 0.009*"để" + 0.009*"thì" + 0.009*"về" + 0.009*"những" + 0.009*"score" + 0.009*"mình" + 0.009*"sẽ" + 0.009*"trả" + 0.009*"có"'), (3, '0.154*"run" + 0.067*"php" + 0.056*"artisan" + 0.056*"queue" + 0.046*"install" + 0.046*"sudo" + 0.046*"batch" + 0.035*"create" + 0.035*"mongodb" + 0.024*"provide"'), (4, '0.076*"query" + 0.055*"functions" + 0.044*"function" + 0.044*"match" + 0.044*"filter" + 0.044*"weight" + 0.044*"test" + 0.034*"boost" + 0.033*"filtering" + 0.023*"matches"')]


In [9]:

import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.104832  0.180887       1        1  40.611007
0     -0.056770 -0.098891       2        1  21.135122
4     -0.101828 -0.117669       3        1  19.471373
3      0.251962  0.003982       4        1  18.701861
2      0.011468  0.031691       5        1   0.080637, topic_info=         Term       Freq      Total Category  logprob  loglift
110       run  11.000000  11.000000  Default  30.0000  30.0000
24      score  10.000000  10.000000  Default  29.0000  29.0000
108       php   5.000000   5.000000  Default  28.0000  28.0000
72       điểm   8.000000   8.000000  Default  27.0000  27.0000
14   function   9.000000   9.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
58        sau   0.002588   2.367213   Topic5  -4.7445   0.3046
45       demo   0.002588   2.367250   Topic5  -4.7445   0.3045
42        chữ   0.002588   2.367313   Topic5  -4.7445   0.3045
49        kết   0.002588   2.367415   Topic5  -4.7446   0.3044
39        cho   0.002588   2.367375   Topic5  -4.7446   0.3044

[244 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
80        2  0.922348  ability
81        2  0.922320     also
4         2  0.343358    apply
4         3  0.686715    apply
102       4  0.912238  artisan
...     ...       ...      ...
28        3  0.556320   weight
114       4  0.937769     work
72        1  0.947620     điểm
73        1  0.844773      đây
74        1  0.974914       để

[141 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 5, 4, 3])